<a href="https://colab.research.google.com/github/housemLassoued/ML-deployment/blob/main/Store_Item_Demand_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


In [ ]:
df=pd.read_csv('/content/train.csv')

In [ ]:
df.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    913000 non-null  object
 1   store   913000 non-null  int64 
 2   item    913000 non-null  int64 
 3   sales   913000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 27.9+ MB


In [ ]:
for col in df.columns:
  print(col,df[col].unique())


date ['2013-01-01' '2013-01-02' '2013-01-03' ... '2017-12-29' '2017-12-30'
 '2017-12-31']
store [ 1  2  3  4  5  6  7  8  9 10]
item [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50]
sales [ 13  11  14  10  12   9   7   5  16  18  15   8   6  21  20  17  22  19
  24  23  26  25  27  38  34  29  31  30   4  36  28  33  32  37  35  43
  40  41  39  42  50  45  44  47  53  49  46  48  51  58  54  57  55  52
   3   2   1  59  56  60  63  69  64  67  65  71  61  73  62  82  78  68
  74  70  87  66  77  88  76  75 102  72  92  86  79  85  81  90  84  80
 103  97  96  95  89 104  94 100  91  83 106 101  98 115  93 111 119  99
 108 110 120 105 126 109 114 113 112 121 107 117 118 139 124 131 123 138
 134 127 136 116 125 122 128 150 129 135 137 132 133 145 130 144   0 148
 141 140 152 147 169 156 159 153 142 157 155 163 143 154 165 146 160 158
 151 164 171 161 177 162 175 181 168 167 149 174 1

In [ ]:
df['date']=pd.to_datetime(df['date'])

In [ ]:
df.describe()

,date,store,item,sales
count,913000,913000.000000,913000.000000,913000.000000
mean,2015-07-02 11:59:59.999999744,5.500000,25.500000,52.250287
min,2013-01-01 00:00:00,1.000000,1.000000,0.000000
25%,2014-04-02 00:00:00,3.000000,13.000000,30.000000
50%,2015-07-02 12:00:00,5.500000,25.500000,47.000000
75%,2016-10-01 00:00:00,8.000000,38.000000,70.000000
max,2017-12-31 00:00:00,10.000000,50.000000,231.000000
std,NaN,2.872283,14.430878,28.801144


In [ ]:
# Execute the code to create the 'month' column first
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

# Now try the groupby operation again, passing the grouping columns as a list
dataset = df.groupby(['year', 'month','store','item'])['sales'].mean() # Removed extra space before this line
# Calculate the mean sales for each year, month, store, and item
dataset = df.groupby(['year', 'month', 'store', 'item'])['sales'].mean().reset_index()

# Function to find the product with max and min sales for each combination of year, month, and store
def find_max_min_sales(group):
    max_row = group.loc[group['sales'].idxmax()]
    min_row = group.loc[group['sales'].idxmin()]
    return pd.Series({
        'year': max_row['year'],
        'month': max_row['month'],
        'store': max_row['store'],
        'item_max_sales': max_row['item'],
        'max_sales': max_row['sales'],
        'item_min_sales': min_row['item'],
        'min_sales': min_row['sales']
    })

# Group again by year, month, and store to find the product with max and min sales
result = dataset.groupby(['year', 'month', 'store']).apply(find_max_min_sales).reset_index(drop=True)

# Display the result
print(result)

       year  month  store  item_max_sales  max_sales  item_min_sales  \
0    2013.0    1.0    1.0            28.0  46.709677             5.0   
1    2013.0    1.0    2.0            15.0  64.161290             5.0   
2    2013.0    1.0    3.0            15.0  58.322581             5.0   
3    2013.0    1.0    4.0            15.0  52.806452             5.0   
4    2013.0    1.0    5.0            13.0  37.903226             5.0   
..      ...    ...    ...             ...        ...             ...   
595  2017.0   12.0    6.0            28.0  56.451613             5.0   
596  2017.0   12.0    7.0            13.0  52.225806             5.0   
597  2017.0   12.0    8.0            15.0  91.806452             5.0   
598  2017.0   12.0    9.0            28.0  79.741935             5.0   
599  2017.0   12.0   10.0            15.0  85.774194             5.0   

     min_sales  
0     8.903226  
1    13.548387  
2    11.387097  
3    10.741935  
4     7.709677  
..         ...  
595  12.580645  

In [ ]:
df['day_name'] = df['date'].dt.day_name()
df['weekday'] = df['day_name'].apply(lambda x: 1 if x in ['Monday', 'Tuesday', 'Wednesday', 'Thursday'] else 0)


In [ ]:
df.head()

,date,store,item,sales,year,month,day,day_name,weekday
0,2013-01-01,1,1,13,2013,1,1,Tuesday,1
1,2013-01-02,1,1,11,2013,1,2,Wednesday,1
2,2013-01-03,1,1,14,2013,1,3,Thursday,1
3,2013-01-04,1,1,13,2013,1,4,Friday,0
4,2013-01-05,1,1,10,2013,1,5,Saturday,0


In [ ]:
import holidays
india_holidays = holidays.country_holidays('IN')


def is_holiday(x):
    if x in india_holidays:
        return 1
    else:
        return 0


df['holidays'] = df['date'].apply(lambda x: is_holiday(x))




In [ ]:
df.head()

,date,store,item,sales,year,month,day,day_name,weekday,holidays
0,2013-01-01,1,1,13,2013,1,1,Tuesday,1,0
1,2013-01-02,1,1,11,2013,1,2,Wednesday,1,0
2,2013-01-03,1,1,14,2013,1,3,Thursday,1,0
3,2013-01-04,1,1,13,2013,1,4,Friday,0,0
4,2013-01-05,1,1,10,2013,1,5,Saturday,0,0


In [ ]:
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'


df['season'] = df['date'].apply(get_season)


In [ ]:
df.head()

,date,store,item,sales,year,month,day,day_name,weekday,holidays,season
0,2013-01-01,1,1,13,2013,1,1,Tuesday,1,0,Winter
1,2013-01-02,1,1,11,2013,1,2,Wednesday,1,0,Winter
2,2013-01-03,1,1,14,2013,1,3,Thursday,1,0,Winter
3,2013-01-04,1,1,13,2013,1,4,Friday,0,0,Winter
4,2013-01-05,1,1,10,2013,1,5,Saturday,0,0,Winter


In [42]:
lags=[30, 60 ,90, 120, 150 , 365 , 395 ,465 , 765 ]
alpha=[0.95 , 0.90 , 0.85 , 0.80 ,0.5 , 0.30 , 0.20 , 0.10]
for lag in lags:
    for a in alpha:
        col_name = 'sales_lag_'+str(lag)+'_alpha_'+str(a)
        df[col_name] = df.groupby(['store', 'item']).sales.apply(lambda x: x.shift(lag)).ewm(alpha=a).mean().reset_index(drop=True)
        df[col_name] = df[col_name].fillna(0)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 83 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   date                      913000 non-null  datetime64[ns]
 1   store                     913000 non-null  int64         
 2   item                      913000 non-null  int64         
 3   sales                     913000 non-null  int64         
 4   year                      913000 non-null  int32         
 5   month                     913000 non-null  int32         
 6   day                       913000 non-null  int32         
 7   day_name                  913000 non-null  object        
 8   weekday                   913000 non-null  int64         
 9   holidays                  913000 non-null  int64         
 10  season                    913000 non-null  object        
 11  sales_lag_30_alpha_0.95   913000 non-null  float64       
 12  sa

In [47]:
df=pd.get_dummies(df,columns=['day_name','season'])

In [49]:
df.drop(['date'],axis=1,inplace=True)

In [50]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
scaler=MinMaxScaler(feature_range=(0,1))
df['sales']=scaler.fit_transform(np.array(df))

In [51]:

X=df.drop(['sales'],axis=1)
y=df['sales']

In [52]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


In [53]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor()
model.fit(x_train,y_train)


RandomForestRegressor()

In [55]:
from sklearn.metrics import mean_squared_error,r2_score
y_pred=model.predict(x_test)
mse=mean_squared_error(y_test,y_pred)
r2=r2_score(y_test,y_pred)

print(mse)
print(r2)

7.233264230948692e-26
1.0
